#### 執行環境(可跳過)

此檔案可上傳至google colab使用，與本地端使用之差異僅於下方資料輸入部分，調整方式見下方註解(使用此方式取得資料之速度較慢) \
使用本機環境，由於公司電腦未安裝python，且因個人習慣，選用anaconda 搭建env使用，以達到各執行續不會因調用同一驅動檔案而出錯之目的，anaconda 安裝方法請見網路上說明，\
env 設定方式:
conda create --name "my_env" python=3.11

安裝:
1. pip
2. ipykernel
3. 所有以下import部分需要的封包

於jupyter notebook中使用env:\
python -m ipykernel install --user --name m_yenv --display-name "Python (myenv)"

---------------------------------------------------------------------------------------------------------------------

## 此檔案為主要執行程序 (用於104網站)

結果取得流程如下:
1. 透過下方未註解部分取得大部分公司資訊
2. 使用find_diff檔案得到未成功取得資料之公司名單
3. 使用此檔案中的註解部分取得結構不同之公司資訊
4. 可再次使用find_diff找出仍無法取得資訊之公司(避免出現於前一步驟已得到資料之公司再次出現於名單中，可改變讀入的名單檔案，或是將with open(record_path,'a') as f中的 'a '改為' w'，如: with open(record_path,'w') as f
5. 剩餘無法成功以此方式取得之公司，扣除公司已不存在之部分，數量極少，若仍欲取得資料可選用人工或UiPath之方式

#### read the name of all the company from a .txt file

若此檔案於colab上執行，需消除下方註解

In [14]:
## only use when using colab
# from google.colab import files
# uploaded = files.upload()

讀取本地端檔案 (若確定檔案內容無誤可跳過此步驟)

In [15]:
with open('company_name_list.txt', 'r') as f:
    lines = f.readlines()
    for i in range(len(lines)):
        print(lines[i].strip())

老爺燒肉高雄店
木棉花國際股份有限公司
新數字有限公司
菫年股份有限公司
極嘉股份有限公司
登豐百岳生活休閒股份有限公司
曉暘電能科技股份有限公司
原醫管理顧問股份有限公司
杜益股份有限公司
台灣天虎國際物流股份有限公司
十易不動產經紀有限公司
巴塞隆納國際股份有限公司
澤峰國際供應鏈股份有限公司
鼎翌餐飲有限公司
星眾科技股份有限公司
臺灣奢茂科技股份有限公司
達文股份有限公司
奧力維智能有限公司
千萬極有限公司
能創半導體股份有限公司
外語吧教育科技股份有限公司
開揚營造股份有限公司
神話休閒股份有限公司
宏力移動股份有限公司
相信光投資股份有限公司
將盛股份有限公司
澳碼科技有限公司
緲思網絡科技有限公司
威士國際股份有限公司
同賞宴股份有限公司
磐旭智能股份有限公司
承鼎餐飲股份有限公司
金曲獎音樂股份有限公司
丰厚國際有限公司
沃盛管理顧問股份有限公司
德騏科技有限公司
米?國際股份有限公司
龍祥鴻資訊有限公司
越創科技股份有限公司
我得有機股份有限公司
御鼎保全股份有限公司
華葳集成股份有限公司
鍇睿國際數位股份有限公司
麥絲瑪拉台灣零售股份有限公司
航空城建築開發股份有限公司
鴿子茶飲股份有限公司
睿智創新科技股份有限公司
博凡星國際有限公司
麟裕國際實業股份有限公司
福滿滿股份有限公司
克林學苑股份有限公司
前電科創股份有限公司
禾米國際食品股份有限公司
常春藤數位出版股份有限公司
極合教育科技股份有限公司
騰齊科技有限公司
台灣應用晶體股份有限公司
宏茂開發股份有限公司
九六八數位資訊股份有限公司
日日加有限公司
耘角家居有限公司
馥境餐廳有限公司
天富餐飲管理顧問股份有限公司
台灣德奈澈貿易股份有限公司
元頂國際控股股份有限公司
群宏國際有限公司
台灣日和藥品有限公司
新哈林運動股份有限公司
好來屋公寓大廈管理維護股份有限公司
定穎投資控股股份有限公司
環球數位金融科技有限公司
方見科技有限公司
永雋科技股份有限公司
租寓平台股份有限公司
阿爾發國際公寓大廈管理維護股份有限公司
卓光科技有限公司
勁捷航空股份有限公司
翊芯科技股份有限公司
低碳鋁材科技股份有限公司
高亨電子科技股份有限公司
泛泰石英股份有限公司
翰可能源股份有限公司
解放生活股份有限公司
樺康智雲股份有限公司
山恩實業股份有限公司
卡諾台灣車業股份有限公司
精靈汽車金融科技股份有

#### import the required packages

In [16]:
import time
import datetime
import requests
import re
import json
from bs4 import BeautifulSoup
import pandas as pd
from datetime import datetime
import os
from openpyxl import Workbook
from openpyxl.utils.dataframe import dataframe_to_rows
import urllib.parse
from urllib.parse import quote, unquote, urljoin, urlparse, parse_qs, urlencode, urlunparse
import csv

### 主要功能部分

由於無法直接從搜尋連結的重組或編碼調整得到公司介紹頁面之資訊，因此將流程劃分為以下步驟:
1. 透過爬取搜尋頁面得之內容，取得第一部分之url
2. 加入後段url資訊以取得公司介紹頁面資料(為避免多次連線造成502錯誤或誤判為惡意攻擊，於下方調整成得到完整url後僅單次連線並暫存回傳內容，於測試單項功能時，可移除註解並調整輸入參數'soup'為欲測試之url)
3. 判斷公司名稱與輸入之公司名稱是否相符
4. 透過id與class選定需要的項目
5. 取得工作機會資訊(於此處所得資訊，經與網頁中元素及所包含的超連結對比，為同一物件，然而，可能會出現數值與網頁上資訊有所出入的情況，此問題暫時無法解決，目前UIPath於104網站資料蒐集已可使用，可用於解決此問題）
6. 將資料存入csv檔中(檔案格式可調整，於此暫時選用csv作為儲存格式)


-----------------------------------------------------------------------------------------------------------------------

下方程式碼為第一次執行所使用

1. https://www.104.com.tw/company/search/?jobsource=cs_n104bank2&keyword={keyword} 為104網站搜索公司之連結，調整其中的keyword以搜尋不同公司，於此為避免因為傳遞時編碼造成回傳資料有誤，對關鍵字先進行url encode
2. 於大部分未顯示‘已關閉‘之公司，其介紹頁面連結部分至於下放class = 'd-md-none company-name-link--mobile jb-link jb-link-blue jb-link-blue--visited font-weight-bold'的元素中


In [17]:
# ## get the url of the company's first page
# def get_info_url(keyword):
#     # url encode
#     encoded_keyword = quote(keyword)
#     href = ''
#     # url = f'https://www.104.com.tw/jobs/search/?jobsource=index_s&keyword={keyword}&mode=s&page=1'
#     url = f'https://www.104.com.tw/company/search/?jobsource=cs_n104bank2&keyword={encoded_keyword}'
#     res = requests.get(url)
#     if res.status_code == 200:
#       print(res.status_code)
#       soup = BeautifulSoup(res.text, "html.parser")
#       parent_div = soup.find('div', id='app')
#       if parent_div:
#         # Find the child <div> within the parent <div> by ID
#         child_div = parent_div.find('div', id='index')
#         if child_div:
#           class_name = 'd-md-none company-name-link--mobile jb-link jb-link-blue jb-link-blue--visited font-weight-bold'
#           a_tag = soup.find('a',class_=class_name)
#           if a_tag and 'href' in a_tag.attrs:
#             # Extract the href attribute
#             href = a_tag['href']
#             return href
#           else:
#             print(f"No <a> tag found with class: {class_name} or no href attribute present.")
#             return None
#         else:
#                 print(f"No child <div> found with ID: 'index'")
#                 return None
#       else:
#           print(f"No parent <div> found with ID: 'app'")
#           return None
#     else:
#         print(f"Failed to retrieve the webpage. Status code: {res.status_code}")
#         return None

# # hr = get_info_url('新數字')
# # print(hr)

下方程式碼為比對公司名單後所使用，僅需將其註解取消，並將上方區塊註解即可使用，其餘部分均相同(須注意一併調整讀入及輸出之檔案)

此部分所使用的搜尋連結與上方區塊相同，唯一不同處為，適用於此區塊之公司於網頁搜尋時大多顯示'已關閉'，因此於前端頁面元素設定或有不同，大多置於 class = 'row no-gutters bg-white rounded overflow-hidden company-lists__summary-wrapper' 的元素中

In [18]:
def get_info_url(keyword):
    # URL encode the keyword
    encoded_keyword = quote(keyword)
    url = f'https://www.104.com.tw/company/search/?jobsource=cs_n104bank2&keyword={encoded_keyword}'

    try:
        res = requests.get(url)
        res.raise_for_status()  # Raise an exception for HTTP errors
        print(res.status_code)

        soup = BeautifulSoup(res.text, "html.parser")
        parent_div = soup.find('div', id='app')

        if parent_div:
            # Find the <div> containing the company link
            child_div = parent_div.find('div', class_='row no-gutters bg-white rounded overflow-hidden company-lists__summary-wrapper')

            if child_div:
                # Find the first <a> tag within the div
                a_tag = child_div.find('a')
                if a_tag and 'href' in a_tag.attrs:
                    # Extract the href attribute
                    href = a_tag['href']
                    return href
                else:
                    print(f"No <a> tag found with href attribute.")
                    return None
            else:
                print(f"No child <div> found with the specified class.")
                return None
        else:
            print(f"No parent <div> found with ID: 'app'")
            return None
    except requests.RequestException as e:
        print(f"Request failed: {e}")
        return None

# Example usage
# hr = get_info_url('永誠工業股份有限公司')
# print(hr)


----------------------------------------------------------------------------------------------------------------------

In [19]:
# get the full url to the info page
def transform_url(original_url, new_params):
    # Parse the original URL
    parsed_url = urlparse(original_url)
    path_parts = parsed_url.path.strip('/').split('/')
    base_url = '/'.join(path_parts[:-1])
    identifier = path_parts[-1]
    query_string = urlencode(new_params, doseq=True)

    new_url = urlunparse((
        parsed_url.scheme,
        parsed_url.netloc,
        f'{base_url}/{identifier}',
        parsed_url.params,
        query_string,
        parsed_url.fragment
    ))

    return new_url

於此區塊對公司名稱進行比對，若相同則回傳輸入之公司名稱，不相同則回傳搜尋所得之公司名稱

In [20]:

def get_name(soup, name):
    # response = requests.get(base_url)
    # if response.status_code == 200:
    #     soup = BeautifulSoup(response.text, 'html.parser')

        # Find the parent div with id 'master'
        parent_div = soup.find('div', id='master')
        if parent_div:
            # Find all <h1> tags
            h1_tags = parent_div.find_all('h1')
            # Extract text from all <h1> tags and concatenate
            compnames = [tag.get_text(strip=True) for tag in h1_tags]
            # Join all names into a single string
            full_text = ' '.join(compnames)
            print(f"Extracted company names: {full_text}")
            # Check if the name is in the full_text
            if name in full_text:
                # return True
                return full_text
            else:
                return full_text
        else:
            print("Can't find the div with the id 'master'")
            return False

# Example usage
# base_url = 'https://www.104.com.tw/company/1a2x6bmrmo?jobsource=cs_n104bank2&roleJobCat%25EF%25BC%259D=company-mobile&roleJobCat='
# result = get_name(base_url, '曉暘電能科技股份有限公司')
# print(result)


所欲取得之內容至於id名稱為‘intro‘之元素的子元素中，於測試時發現主要內容（產業類別、產業描述等資訊）均位於第二子元素中，因此將其中所有內容取出

In [21]:
#get the element inside the info page
def get_all_children(soup, div_id):
    # response = requests.get(base_url)
    # if response.status_code == 200:
    #     soup = BeautifulSoup(response.text, 'html.parser')

        ## find the element
        parent_div = soup.find('div', id=div_id)
        if parent_div:
            children = parent_div.find_all(recursive=False)  # `recursive=False` gets only direct children
            child_elements = []
            for child in children:
                child_elements.append({
                    'tag': child.name,
                    'text': child.get_text(strip=True),
                    'attributes': child.attrs
                })
            return child_elements
        else:
            print(f"No <div> found with ID: {div_id}")
            return None
    # else:
    #     print(f"Failed to retrieve the webpage. Status code: {response.status_code}")
    #     return None


於此區塊透過選定之可唯一識別之元素id取出其值，也於執行過程中印出此元素內容，可與網頁內容對比，為同一元素，然而，取得數值有時會與網頁上呈現的數值不同，此問題暫時未找到解法

In [22]:

def get_work_opportunity(soup, id='info06'):
    # response = requests.get(baseurl)
    # if response.status_code == 200:
    #     soup = BeautifulSoup(response.text, 'html.parser')

        # Find the parent div with id 'master'
        parent_div = soup.find('div', id='master')
        if parent_div:
            # Find the div with class 'header-menu__container'
            container_div = parent_div.find('div', class_='header-menu__container')
            if container_div:
                # Try searching for the <a> tag with or without the '#' character
                opportunity = container_div.find('a', id=id)
                if not opportunity:
                    opportunity = container_div.find('a', id=f'#{id}')
                    print(opportunity)

                if opportunity:
                    # Extract the text and find the number using regex
                    text = opportunity.get_text(strip=True)
                    return text
                    # print(f"Extracted text: {text}")  # Debugging line
                else:
                    print(f"Can't find the <a> tag with the id: {id} or #{id}")
                    return None
            else:
                print("Can't find the div with the class name 'header-menu__container'")
                return None
        else:
            print("Can't find the div with the id 'master'")
            return None
    # else:
    #     print(f"Request failed with status code: {response.status_code}")
    #     return None

# # Example usage
# base_url = 'https://www.104.com.tw/company/1a2x6bmrmo?jobsource=cs_n104bank2&roleJobCat%25EF%25BC%259D=company-mobile&roleJobCat='
# result = get_work_opportunity(base_url)
# print(result)


於上述取出之第二子元素中可能會存有無用之內容，且其開頭名稱均相同，因此僅保留此關鍵字前的內容

In [23]:
##remove the unuse content part
def remove_after_keyword(text, keyword):
    pattern = re.compile(f'{re.escape(keyword)}.*')
    result = pattern.sub('', text)

    return result.strip()


解析所得字串內容，以關鍵詞作為切分點，並以key,value形式紀錄，以便後續資料儲存

In [24]:
## process the data that got
## i need to split the whole context base on different keywords
def split_by_keywords(content, keywords):
    # Create a regex pattern to split content based on the keywords, keeping the keywords
    keywords_pattern = '|'.join(re.escape(keyword) for keyword in keywords)
    pattern = re.compile(f'({keywords_pattern})')
    parts = [part.strip() for part in pattern.split(content) if part.strip()]

    info = {}
    for i, part in enumerate(parts):
        if i % 2 == 0:
            continue
        keyword = parts[i-1]
        if keyword in keywords:
            info[keyword] = part.strip()

    return info

def extract_info(content):
  # the element that i need to get and save
    keywords = [
        '產業類別', '產業描述', '電話', '傳真', '資本額', '員工人數', '公司網址', '地址', '聯絡人'
    ]
    sections = split_by_keywords(content, keywords)
    info = {key: sections.get(key, 'null') for key in keywords}

    return info


def main(content, company_name, file_name,num_work):
    global 產業類別, 產業描述, 公司電話, 傳真, 資本額, 員工人數, 公司網址, 公司地址, 聯絡人

    info = extract_info(content)

    info['公司名稱'] = company_name
    info['工作機會'] = num_work
    write_to_csv(file_name, info)


In [25]:
## save into a csv file
def write_to_csv(file_name, data):
    header = [
          '公司名稱', '產業類別', '產業描述', '公司電話', '傳真', '資本額', '員工人數', '公司網址', '公司地址', '聯絡人','工作機會'
    ]
    with open(file_name, mode='a', newline='', encoding='utf-8') as file:
        file.write('\ufeff') ##確保透過excel開啟時編碼正確
        writer = csv.writer(file)
        ## check header
        if file.tell() == 0:
            writer.writerow(header)
        writer.writerow([
            data.get('公司名稱', ''),
            data.get('產業類別', ''),
            data.get('產業描述', ''),
            data.get('電話', ''),
            data.get('傳真', ''),
            data.get('資本額', ''),
            data.get('員工人數', ''),
            data.get('公司網址', ''),
            data.get('地址', ''),
            data.get('聯絡人', ''),
            data.get('工作機會', '')
        ])

In [26]:
if __name__ == "__main__":
  csv_file_name = 'result_similar.csv'
  first_url = ''
  full_url = ''
  公司名稱 = ''
  產業類別 = ''
  產業描述  = ''
  公司電話 = ''
  傳真 =''
  資本額 = ''
  員工人數 =''
  公司網址 = ''
  公司地址 =''
  聯絡人 = ''
  工作機會 =''
  # getname = False
  with open('different_new.txt', 'r') as f:
    lines = f.readlines()
    num_work = 0
    for i in range(len(lines)):
    # for i in range(3):
        公司名稱 = lines[i].strip()
        print(f'公司名稱 ：{公司名稱}')
        keyword = lines[i].strip()
        first_url = get_info_url(keyword)
        if first_url:
          print(f"First URL: {first_url}")
          new_params = {
            'jobsource': 'cs_n104bank2',
            'roleJobCat%EF%BC%9D': 'company-mobile',
            'roleJobCat': ''
          }
          full_url = transform_url(first_url, new_params)
          print(f"Full URL: {full_url}")
          if full_url:
            response = requests.get(full_url)
            if response.status_code == 200:
              soup = BeautifulSoup(response.text, 'html.parser')
              getname = get_name(soup,公司名稱)
              公司名稱 = getname
              if getname != None:
                div_id = 'intro'  ## the id of the div that put the info
                children = get_all_children(soup, div_id)
                工作機會 = get_work_opportunity(soup)
                print(f"工作機會:{工作機會}")
                match = re.search(r'\d+[\+\d]*', 工作機會)
                if match:
                  num_work = match.group()
                  print(f"num:{num_work}")
                else:
                  num_work = 0
                  print("Number not found in the text")
                ## the second child is the element that contains all the other info needed
                informaitions = ''
                if children:
                    if len(children) > 1:
                      child_2 = children[1]
                      print("Data in Child 2:")
                      print(f"  Text: {child_2['text']}")
                      informations = child_2['text']
                    else:
                      print("Child 2 is not available in the extracted data.")
                    if informations:
                      #remove the content after keyword
                      keyword = "相關連結"
                      informations = remove_after_keyword(informations, keyword)
                      print(f" Informations {informations}")
                      main(informations, 公司名稱, csv_file_name,num_work)
                      # print(公司地址)
              else:
                print("wrong company name")


公司名稱 ：老爺燒肉高雄店
Failed to retrieve the webpage. Status code: 404
公司名稱 ：木棉花國際股份有限公司
200
First URL: https://www.104.com.tw/company/13xnvodc
Full URL: https://www.104.com.tw/company/13xnvodc?jobsource=cs_n104bank2&roleJobCat%25EF%25BC%259D=company-mobile&roleJobCat=
Extracted company names: 木棉花創投股份有限公司
<a class="d-flex justify-content-center align-items-end flex-shrink-0 py-2 py-md-3 px-4 h3 menu-item" data-gtm-head="工作機會" data-v-b8bc2a94="" href="/company/13xnvodc#info06" id="#info06">工作機會(21)</a>
工作機會:工作機會(21)
num:21
Data in Child 2:
  Text: 產業類別廣播電視節目供應業聯絡人人資產業描述商品肖像、活動展覽授權/影音代理發行電話02-29039101資本額8000萬元經濟部商業司查詢傳真02-29036346員工人數180人地址新北市新莊區思源路555號17樓公司網址https://www.e-muse.com.tw相關連結最新消息動畫介紹LinkedInYoutubeFacebook
 Informations 產業類別廣播電視節目供應業聯絡人人資產業描述商品肖像、活動展覽授權/影音代理發行電話02-29039101資本額8000萬元經濟部商業司查詢傳真02-29036346員工人數180人地址新北市新莊區思源路555號17樓公司網址https://www.e-muse.com.tw
公司名稱 ：新數字有限公司
200
First URL: https://www.104.com.tw/company/1a2x6bmr13
Full URL: https://www.104.com.tw/company/1a2x6bmr13?jobso

KeyboardInterrupt: 